In [33]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torch.nn.utils.rnn import pad_sequence
import random
from torch.utils.data import random_split
# Step 1: Load the data
lineage_file = "./dataset1.csv"
text_files_folder = "./text_files"

# Step 2: Load all the text files to converted_df
converted_data = []
for file_path in os.listdir(text_files_folder):
    file_name = os.path.splitext(file_path)[0]
    with open(os.path.join(text_files_folder, file_path), "r") as file:
        content = file.read().strip()
        converted_data.append(['"' + file_name + '"', content])

converted_df = pd.DataFrame(converted_data, columns=["ID", "Enc_Sequence"])
max_length = converted_df["Enc_Sequence"].str.len().max()
####################################################################################

# Step 3: Load lineage file as DataFrame
lineage_df = pd.read_csv(lineage_file)
print(lineage_df)
merged_df = pd.merge(converted_df, lineage_df, on="ID", how="left")
converted_df["lineage"] = merged_df["lineage"]

print(converted_df)


lineages = lineage_df["lineage"].unique()
num_classes = len(lineages)
lineage_to_id = {lineage: i for i, lineage in enumerate(lineages)}

#####################################################################################

# Step 4: Prepare the data for training
class ViralSequencesDataset(Dataset):
    
    def __init__(self, converted_df, max_length, lineage_to_id):
        self.data = converted_df
        self.max_length = max_length
        self.lineage_to_id = lineage_to_id
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        sequence = self.data.iloc[index]["Enc_Sequence"]
        enc_sequence = np.array(list(sequence), dtype=np.uint8)
        enc_sequence = enc_sequence.reshape(-1)  # Remove extra dimensions

        # Pad the sequence with zeros
        padded_sequence = nn.functional.pad(torch.tensor(enc_sequence), pad=(0, self.max_length - len(enc_sequence)))

        lineage = self.data.iloc[index]["lineage"]
        lineage_id = self.lineage_to_id[lineage]

        return padded_sequence, lineage_id, len(enc_sequence)  # Return sequence length
    
    def __str__(self):
        dataset_info = f"ViralSequencesDataset\nDataset size: {len(self)}\nMax length: {self.max_length}"
        data_info = f"Data:\n{self.data.head()}"
        return f"{dataset_info}\n\n{data_info}"


# dataset = ViralSequencesDataset(output_file, max_length)
dataset = ViralSequencesDataset(converted_df, max_length, lineage_to_id)
print(dataset)
# Set the batch size and split ratio
batch_size = 64
train_ratio = 0.8

train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
#####################################################################################

# Step 5: Define the model architecture
class LineageClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LineageClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_classes)

    def forward(self, x, length):
        out = self.fc1(x.float())
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out
#####################################################################################

# Step 6: Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Set the sequence length
sequence_length = max_length

# Set the number of output classes
num_classes = 10  # Change this to the appropriate number of classes

Hidden_size=512

# Initialize the model
model = LineageClassifier(input_size=sequence_length, hidden_size=Hidden_size, num_classes=num_classes)
model = model.to(device)

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Set the number of epochs
num_epochs = 5

# Initialize variables for early stopping
best_loss = float('inf')
patience = 3
counter = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0

    for batch_idx, (inputs, targets, lengths) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, lengths)  # Pass lengths to the model

        targets = targets.view(-1)
        outputs = outputs.view(-1, num_classes)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == targets).sum().item()

        if batch_idx % 10 == 9:
            batch_loss = running_loss / 10
            accuracy = 100 * correct_predictions / (10 * len(inputs))
            print(f"Epoch [{epoch+1}/{num_epochs}] - Batch [{batch_idx+1}/{len(train_loader)}] - Loss: {batch_loss:.4f} - Accuracy: {accuracy:.2f}%")
            running_loss = 0.0
            correct_predictions = 0

#####################################################################################

# Step 7: Save the trained model
torch.save(model.state_dict(), "./lineage_classifier.pth")


              ID    lineage
0     "ON414702"  "BA.2.31"
1     "ON056475"  "BA.2.31"
2     "ON203732"  "BA.2.31"
3     "OW275801"  "BA.2.31"
4     "ON914730"  "BA.2.31"
...          ...        ...
4995  "OM122217"  "BA.1.17"
4996  "OV662626"  "BA.1.17"
4997  "OM200680"  "BA.1.17"
4998  "OW742696"  "BA.1.17"
4999  "OW120613"  "BA.1.17"

[5000 rows x 2 columns]
              ID                                       Enc_Sequence    lineage
0     "FR990446"  0000010000000101000001010000011111110110001101...  "B.1.258"
1     "FR991388"  1010111111001100010111110101010010101100000100...  "B.1.258"
2     "FR991439"  0000010100000111111101100011011101111110110010...  "B.1.258"
3     "HG994296"  0000000000000000000000000000000000000001010010...  "B.1.258"
4     "HG994312"  0000000000000000000000000000000000000000000000...  "B.1.258"
...          ...                                                ...        ...
4995  "OX273314"  0000000000000000000000000000000000000000000000...   "BA.5.1"
4996  "

In [34]:

#####################
# WORKING TEST
#####################
# Step 8: Test the model
def test_model(model, test_loader, device):
    model.eval()
    total_samples = 0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, targets, lengths in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs, lengths)
            _, predicted = torch.max(outputs.data, 1)

            total_samples += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

    accuracy = 100 * correct_predictions / total_samples
    print(f"Test Accuracy: {accuracy:.2f}%")

# Call the test function
test_model(model, test_loader, device)

Test Accuracy: 76.30%


In [1]:
import random
import requests
import io
import re
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO
import concurrent.futures

with open("./data/metadata", "r") as file:
    metadata_lines = file.readlines()

lineage_samples = defaultdict(list)
unique_lineages = set()
samples_list = []

for line in metadata_lines[1:]:
    columns = line.strip().split("\t")
    accession_id = columns[0]
    lineage = columns[1]
    lineage_samples[lineage].append(accession_id)
    unique_lineages.add(lineage)

# Ensure selected lineages have at least 500 samples
selected_lineages = [lineage for lineage in unique_lineages if len(lineage_samples[lineage]) >= 500][:10]

max_sequences_per_lineage = 500
max_sequences = max_sequences_per_lineage * len(selected_lineages)

def download_sample(sample, lineage):
    url = f"https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ena_sequence&id={sample}&format=fasta&style=raw&Retrieve=Retrieve"
    response = requests.get(url)
    if response.status_code == 200:
        filename = re.sub(r'[\\/:\*\?"<>\|()]', '_', sample)
        with io.open(f"./data/fasta_files/{filename}.fasta", "w", encoding="utf-8") as seq_file:
            seq_file.write(response.text)
        # Append the data to the list without the sequence
        return {'id': sample, 'lineage': lineage}

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for lineage in tqdm(selected_lineages, desc="Processing lineages"):
        samples = random.sample(lineage_samples[lineage], max_sequences_per_lineage)
        lineage_samples[lineage] = list(set(lineage_samples[lineage]) - set(samples))
        futures = {executor.submit(download_sample, sample, lineage) for sample in samples}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                samples_list.append(result)

print("Download completed!")

# Convert the list of dictionaries to a dataframe
df = pd.DataFrame(samples_list)

# Save the dataframe to csv
df.to_csv("./data/dataset.csv", index=False)

Processing lineages: 100%|█████████████████████████████████████████████████████████████| 10/10 [02:25<00:00, 14.58s/it]

Download completed!


In [2]:
# model.py
import torch
from torch import nn

class DNA_CNN(nn.Module):
    def __init__(self):
        super(DNA_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(4, 1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(4, 1))
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 10)
        self.max_pool = nn.MaxPool2d((2, 1))
        self.adaptive_pool = nn.AdaptiveMaxPool2d((1, 4))
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [2]:
import warnings
from CNN2D import DNA_CNN
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm

warnings.filterwarnings("ignore")


def load_data():
    data_path = "./data/processed_data.npz"

    if os.path.exists(data_path):
        # Load the sequences and labels from the .npz file if it already exists
        data = np.load(data_path, allow_pickle=True)
        sequences = torch.Tensor(data['sequences'])
        labels = torch.LongTensor(data['labels'])
    else:
        sequences = []
        labels = []
        reduced_data = pd.read_csv("./data/reduced_data_encoded.csv")
        reduced_data["id"] = reduced_data["id"].str.strip('"')
        npy_files = os.listdir("./data/encoded_sequences/")
        for npy_file in npy_files:
            id = npy_file.split(".")[0]
            sequence = np.load(f"./data/encoded_sequences/{npy_file}")
            label = reduced_data.loc[reduced_data["id"] == id, "label"].values
            sequences.append(sequence)
            labels.append(label)

        # Find the maximum and minimum sequence lengths
        seq_lengths = [len(seq) for seq in sequences]
        max_length = max(seq_lengths)
        min_length = min(seq_lengths)
        target_length = (max_length + min_length) // 2

        # Pad or truncate sequences to match the target length
        for i in range(len(sequences)):
            difference = target_length - len(sequences[i])
            if difference > 0:  # Sequence is shorter than target length
                padding = np.repeat([[0.25, 0.25, 0.25, 0.25]], difference, axis=0)
                sequences[i] = np.concatenate([sequences[i], padding])
            elif difference < 0:  # Sequence is longer than target length
                sequences[i] = sequences[i][:target_length]

        # Save the processed sequences and labels to a .npz file
        np.savez(data_path, sequences=sequences, labels=labels)

        sequences = torch.Tensor(sequences)
        labels = torch.LongTensor(labels)

    return sequences, labels

def train(model, criterion, optimizer, train_loader, device):
    model.train()  # set the model to training mode
    total_loss = 0.0
    for i, (sequences, labels) in enumerate(tqdm(train_loader)):
        sequences = sequences.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss



def evaluate(model, test_loader,device):
    model.eval()
    total = 0
    correct = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in tqdm(test_loader, desc="Evaluating"):
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test sequences: {} %'.format((correct / total) * 100))
    return torch.tensor(all_predictions), torch.tensor(all_labels)


def main():
    data = np.load("./data/processed_data.npz", allow_pickle=True)
    sequences = torch.Tensor(data['sequences'])
    labels = torch.LongTensor(data['labels'])
    print(sequences.shape)
    print(labels.shape)

    sequences = sequences.unsqueeze(1)
    labels = labels.squeeze()

    print(sequences.shape)
    print(labels.shape)

    # sequences, labels = load_data()
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

    train_data = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_data, batch_size=32)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if device.type == 'cuda':
        print('Running on GPU: ', torch.cuda.get_device_name(0))  # Print the name of the GPU
    else:
        print("Running on CPU")

    model = DNA_CNN()
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    avg = train(model, criterion, optimizer, train_loader, device)
    print(avg)

    test_data = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_data, batch_size=32)
    predicted, test_labels = evaluate(model, test_loader,device)

    print(classification_report(test_labels.cpu().numpy(), predicted.cpu().numpy()))


if __name__ == "__main__":
    main()


torch.Size([5000, 28821, 4])
torch.Size([5000, 1])
torch.Size([5000, 1, 28821, 4])
torch.Size([5000])
Running on CPU


  6%|█████▏                                                                            | 8/125 [00:49<11:58,  6.14s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\roeym\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\roeym\AppData\Local\Temp/ipykernel_22440/223147076.py", line 145, in <module>
    main()
  File "C:\Users\roeym\AppData\Local\Temp/ipykernel_22440/223147076.py", line 134, in main
    avg = train(model, criterion, optimizer, train_loader, device)
  File "C:\Users\roeym\AppData\Local\Temp/ipykernel_22440/223147076.py", line 73, in train
    loss.backward()
  File "C:\Users\roeym\anaconda3\lib\site-packages\torch\_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "C:\Users\roeym\anaconda3\lib\site-packages\torch\autograd\__init__.py", line 173, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above

TypeError: object of type 'NoneType' has no len()

In [10]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


False


In [9]:
import torch
print(torch.__version__)


1.12.1


In [11]:
import cuda
print(cuda.__version__)


ModuleNotFoundError: No module named 'cuda'